# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weather_df= pd.read_csv("weather_df.csv")
weather_df= weather_df.rename(columns={"Latitude":"Lat",
                  "Longitude":"Lng",
                    "City Name": "City"})
weather_df

,City,City ID,Lat,Lng,Temp °F,Humidity %,Wind Speed (mph),Cloudiness %,Country
0,Arraial do Cabo,3471451,-22.9661,-42.0278,78.80,83,10.36,75,BR
1,Turinsk,1488933,58.0458,63.6960,34.21,97,12.03,100,RU
2,Gdov,561679,58.7406,27.8122,4.73,94,4.47,90,RU
3,Busselton,2075265,-33.6500,115.3333,57.99,78,1.01,100,AU
4,Jamestown,5122534,42.0970,-79.2353,27.00,63,21.85,90,US
...,...,...,...,...,...,...,...,...,...
554,Necochea,3430443,-38.5473,-58.7368,82.71,24,6.42,0,AR
555,Kavaratti,1267390,10.5669,72.6420,80.40,77,12.95,0,IN
556,The Valley,3573374,18.2170,-63.0578,82.40,65,16.11,20,AI
557,Karaton,1630058,-6.3424,106.1045,80.60,78,9.22,75,ID


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
query_url= f"{base_url}{g_key}"
fig = gmaps.figure(zoom_level= 2, center=(37,-95))
gmaps.configure(api_key=g_key)
locations = weather_df[["Lat", "Lng"]].astype(float)
Humidity= weather_df["Humidity %"]
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
weather_df

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#A max temperature lower than 80 degrees but higher than 70. Criteria 1
#Wind speed less than 10 mph. Criteria 2
#Zero cloudiness. Criteria 3
step1= weather_df.loc[(weather_df["Temp °F"] >= 70) & (weather_df["Temp °F"] <= 80)]
step2= step1.loc[step1["Wind Speed (mph)"] <= 10]
hotel_df= step2.loc[step2["Cloudiness %"] == 0]
hotel_df

,City,City ID,Lat,Lng,Temp °F,Humidity %,Wind Speed (mph),Cloudiness %,Country
36,Saint-Philippe,935215,-21.3585,55.7679,78.80,65,5.75,0,RE
43,Chake Chake,160677,-5.2459,39.7666,78.19,83,6.17,0,TZ
78,Pathein,1328421,16.7833,94.7333,70.36,91,6.53,0,MM
79,Upington,945945,-28.4478,21.2561,77.00,22,3.44,0,ZA
98,Castro,3466704,-24.7911,-50.0119,72.12,62,4.16,0,BR
110,San Vicente,3428068,-26.6167,-54.1333,76.10,40,7.14,0,AR
114,Am Timan,245338,11.0297,20.2827,75.00,19,7.45,0,TD
116,Obo,236950,5.3998,26.4869,76.93,20,6.51,0,CF
199,Mar del Plata,3430863,-38.0023,-57.5575,75.20,46,9.22,0,AR
355,Illapel,3887344,-31.6308,-71.1653,77.76,37,7.94,0,CL


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
url= "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
hotel_df["Hotel Name"]= ""
for index, row in hotel_df.iterrows():
    lat=str(row['Lat'])
    lng=str(row['Lng'])
#     print(lat)
#     print(lng)
    query_url= f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=5000&type=lodging&key={g_key}"
    response=requests.get(query_url).json()
#     print(response)
#There is no hotel type
    try:
        hotel_df.loc[index, "Hotel Name"]= response['results'][0]['name']
    except:
        print("Missing result: skipping..")
hotel_df

C:\Users\PhuongVu\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\PhuongVu\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing result: skipping..


,City,City ID,Lat,Lng,Temp °F,Humidity %,Wind Speed (mph),Cloudiness %,Country,Hotel Name
36,Saint-Philippe,935215,-21.3585,55.7679,78.80,65,5.75,0,RE,"Chambres d'hôte ""La Trinité"""
43,Chake Chake,160677,-5.2459,39.7666,78.19,83,6.17,0,TZ,Hotel Archipelago
78,Pathein,1328421,16.7833,94.7333,70.36,91,6.53,0,MM,Pathein Hotel
79,Upington,945945,-28.4478,21.2561,77.00,22,3.44,0,ZA,River Place Manor
98,Castro,3466704,-24.7911,-50.0119,72.12,62,4.16,0,BR,CHACARA BAILLY
110,San Vicente,3428068,-26.6167,-54.1333,76.10,40,7.14,0,AR,Complejo Turístico Farelluk
114,Am Timan,245338,11.0297,20.2827,75.00,19,7.45,0,TD,Palais du chef Canton Salamt
116,Obo,236950,5.3998,26.4869,76.93,20,6.51,0,CF,
199,Mar del Plata,3430863,-38.0023,-57.5575,75.20,46,9.22,0,AR,Gran Hotel Mar del Plata
355,Illapel,3887344,-31.6308,-71.1653,77.76,37,7.94,0,CL,Hotel Londres


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

# Display figure


IndexError: list index out of range